In [1]:
import numpy
from matplotlib import pyplot
from matplotlib import colors
% matplotlib inline

In [2]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [3]:
import utils.poly as poly
import utils.quadrature as quad

$$
\frac{\partial u}{\partial t} + \frac{\partial u}{\partial x} = 0
$$

$$
u_{init} = e^{-40(x-0.5)^2}
$$

$$
u_{exact} = e^{\left[-40(x-0.5-t)^2\right]}
$$

In [4]:
def equalDistrib(N):
    """"""
    return numpy.linspace(-1., 1., N, endpoint=False) + 1. / N

In [5]:
def LegendreLobatto(N):
    """"""
    return quad.GaussLobattoJacobi(N).nodes

In [6]:
def LegendreGauss(N):
    """"""
    return quad.GaussJacobi(N).nodes

In [7]:
def ChebyshevLobatto(N):
    """"""
    return - numpy.cos(numpy.arange(N, dtype=numpy.float64) * numpy.pi / (N-1))

In [8]:
class Config(object):
    """data structure for configurations"""
    
    def __init__(self, config=None):
        """initialization
        
        Args:
            config: string or file containing configurations
        """
        # TODO: check the type of config
        # TODO: implement reading configurations from config
        
        self.xLB = 0.
        self.xRB = 1.
        self.Ne = 10
        self.K = 4
        self.g_type = "g1"
        self.node_type = "LegendreGauss"
        
        self.BCL = "periodic"
        self.BCR = "periodic"

In [9]:
class Element(object):
    """"""
    
    distrib_pnts = {
        "equal": equalDistrib,
        "LegendreLobatto": LegendreLobatto,
        "LegendreGauss": LegendreGauss,
        "ChebyshevLobatto": ChebyshevLobatto}
    
    def __init__(self, K, xL, xR, distrib="equal"):
        """initialization
        
        Args:
            K: integer, the number of solution points
            xL: float, the coordinate of left boundary
            xR: float, the coordinate of right boundary
            distrib: string, the distribution of solution points (options: 
                    equal, LegendreLobatto, LegendreGauss, ChebyshevLobatto)
        """
        # TODO: check the type of input arguments
        
        self.xL = numpy.float64(xL)
        self.xR = numpy.float64(xR)
        self.h = self.xR - self.xL
        self.xc = 0.5 * (self.xL + self.xR)
        self.J = 0.5 * self.h
        self.Jinv = 2. / self.h
        
        self.K = K
        self.xi_soln_pnts = Element.distrib_pnts[distrib](K)
        self.x_soln_pnts = self.coord_l2g(self.xi_soln_pnts)
        
        self.basis = {
            "local": poly.LagrangeBasis(self.xi_soln_pnts),
            "global": poly.LagrangeBasis(self.x_soln_pnts)}
        
        self.D = self.basis["local"].derivative(self.xi_soln_pnts)

    def coord_l2g(self, xi):
        """coordinate transform from local (xi) to global (x)"""
        return self.xc + self.J * xi

    def coord_g2l(self, x):
        """coordinate transform from global (x) to local (xi)"""
        return self.Jinv * (x - self.xc)

In [10]:
class LocalSolutionPnts(object):
    """data structure for solution points built on a given element"""
    
    def __init__(self, element, flux, ic=numpy.zeros_like):
        """initialization
        
        Args:
            element: an instance of class Element
            flux: flux function defined from PDE
            ic: initialization function, i.e., initial conditions
        """
        # TODO: check the type of input arguments
        
        self.element = element
        
        self.props = {
            "ujk": numpy.empty(self.element.K, dtype=numpy.float64),
            "fjk": numpy.empty(self.element.K, dtype=numpy.float64)}
        
        self.setup_ujk(ic)
        self.calculate_fjk(flux)
    
    def setup_ujk(self, ic=numpy.zeros_like):
        """set up (initialize) the value of u at solution points"""
        self.props["ujk"] = ic(self.element.x_soln_pnts)
    
    def calculate_fjk(self, f):
        """update f_{j, k}
        
        Args:
            f: function of flux from PDE
        """
        self.props["fjk"] = f(self.props["ujk"])
    
    def __call__(self, x, p_type="ujk", c_type="local"):
        """return interpolated u at location x (or xi)
        
        Args:
            x: float, can be global or local (default) coordinates
            p_type: string, "ujk" or "fjk", the properties on solution points
                    that will be used for interpoation
            c_type: string, "local" or "global", specifies the type of 
                    coordinates provided
        """
        return numpy.dot(self.element.basis[c_type](x), self.props[p_type])
    
    def get_dfjk(self):
        """get derivatives of local discontinuous flux at solution points"""
        
        return numpy.dot(self.element.D, self.props["fjk"])

In [11]:
class ElementSet(object):
    """"""
    
    def __init__(self, config):
        """initialization
        
        Args:
            config: an instance of class Config
        """
        
        self.xLB = config.xLB
        self.xRB = config.xRB
        self.Ne = config.Ne
        
        self._init_interface()
        self._init_element()
    
    def _init_element(self):
        """initialize each element"""
        
        self.elements = {i: Element(config.K, self.interface_x[i], 
                                    self.interface_x[i+1], config.node_type)
                         for i in range(self.Ne)}
            
        self.elements[-1] = self.elements[self.Ne-1]
        self.elements[self.Ne] = self.elements[0]
    
    def _init_interface(self):
        """initialize interface information"""
        
        self.interface_x = numpy.linspace(self.xLB, self.xRB, self.Ne+1)
        self.interface_i2e = {i: (i-1, i) for i in range(self.Ne+1)}
        self.interface_e2i = {v: k for k, v in self.interface_i2e.items()}
        self.interface_fupw = numpy.zeros_like(self.interface_x)

    def __getitem__(self, key):
        """override the operator []"""
        return self.elements[key]

    def __setitem__(self, key, value):
        """override the operator [] when writing new values"""
        
        raise NotImplementedError(
            "changing single element using the operator [] is not supported")

    def __delitem__(self, key, value):
        """override the operator [] when writing new values"""
        
        raise NotImplementedError(
            "deleting single element using the operator [] is not supported")

    def __iter__(self):
        """return iterator"""
        return self.elements.__iter__()
    
    def __next__(self):
        """override of __next__"""
        self.elements.__next__()

In [12]:
def u_init(x):
    """initial condition of u"""
    temp = x - 0.5
    return numpy.exp(-40 * temp * temp)

In [13]:
def flux(u):
    """flux in PDE"""
    return u

In [ ]:
def u_exact(x, t):
    """exact solution of u"""
    temp = x - 0.5 - t
    return numpy.exp(-40 * temp * temp)

In [ ]:
def RK4(u, dx, dt, rhs):
    """"""
    k1 = rhs(u, dx, dt)
    k2 = rhs(u + 0.5 * dt * k1, dx, dt)
    k3 = rhs(u + 0.5 * dt * k2, dx, dt)
    k4 = rhs(u + dt * k3, dx, dt)
    u += dt * (k1 + 2. * k2 + 2. * k3 + k4) / 6.
    return u